# Insights Module - Refine

This notebook demonstrates the utility of the OEA_py class notebook, and speeding up the process of refining/pseudonymizing the Insights data.

The steps outlined below describe how this notebook is used to refine the Microsoft Education Insights module tables:

- Set the workspace for where the tables are located. 
- 1 function is defined and used:
   1. **refine_insights_dataset**: uses the Insights metadata.csv to pseudonymize each table according to whether it is to be hashed, masked, or has no-operation.

In [ ]:
workspace = 'dev'

In [ ]:
%run OEA_py

In [ ]:
# 1) set the workspace (this determines where in the data lake you'll be writing to and reading from).
# You can work in 'dev', 'prod', or a sandbox with any name you choose.
# For example, Sam the developer can create a 'sam' workspace and expect to find his datasets in the data lake under oea/sandboxes/sam
oea.set_workspace(workspace)

In [ ]:
# 2) this step refines the data through the use of metadata (this is where the pseudonymization of the data occurs).
def refine_insights_dataset(tables_source):
    items = oea.get_folders(tables_source)
    for item in items: 
        table_path = tables_source +'/'+ item
        if item == 'metadata.csv':
            logger.info('ignore metadata processing, since this is not a table to be ingested')
        else:
            try:
                if item == 'activity':
                    oea.refine('M365/v1.14/activity', metadata[item], 'SignalId')
                elif item == 'AadGroup':
                    oea.refine('M365/v1.14/AadGroup', metadata[item], 'ObjectId_pseudonym')
                elif item == 'AadUser':
                    oea.refine('M365/v1.14/AadUser', metadata[item], 'ObjectId_pseudonym')
                elif item == 'AadUserPersonMapping':
                    oea.refine('M365/v1.14/AadUserPersonMapping', metadata[item], 'ObjectId_pseudonym')
                elif item == 'Person':
                    oea.refine('M365/v1.14/Person', metadata[item], 'Id_pseudonym')
                elif item == 'PersonDemographic':
                    oea.refine('M365/v1.14/PersonDemographic', metadata[item], 'PersonId_pseudonym')
                else:
                    oea.refine('M365/v1.14/' + item, metadata[item], 'Id')
            except AnalysisException as e:
                # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
                pass
            
            logger.info('Refined table: ' + item + ' from: ' + table_path)

In [ ]:
metadata = oea.get_metadata_from_url('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/main/modules/module_catalog/Microsoft_Education_Insights/test_data/metadata.csv')
refine_insights_dataset('stage2/Ingested/M365/v1.14')